
# Análise de Fatores de Risco para a Covid-19 

Neste projeto, serão analisados os fatores de risco para a Covid-19, levando em consideração doenças que são fatores de risco, idade, gênero e outras características.  

**Alunas: Eliane Dantas de Jesus e Natalia da Silva Costa**

### Imports necessários para a execução do projeto

In [18]:
import pandas as pd
import re
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.naive_bayes import GaussianNB, MultinomialNB

### Leitura do arquivo csv

In [19]:
dados = pd.read_csv('Casos_e_obitos_ESP.csv', sep=';')

### Verificação da quantidade de registros

In [20]:
num_registros = len(dados)
print("Número total de registros:", num_registros)

Número total de registros: 100000


### Verificação dos valores faltando por coluna

In [21]:
valores_faltando = dados.isnull().sum()
print("\nValores faltando por coluna:")
print(valores_faltando)


Valores faltando por coluna:
Asma                          0
Cardiopatia                   0
Data_Inicio_Sintomas       3569
Diabetes                      0
Diagnostico_Covid19           0
Doenca_Hematologica           0
Doenca_Hepatica               0
Doenca_Neurologica            0
Doenca Renal                  0
Genero                        0
Idade                      3198
Imunodepressao                0
Municipio                     0
Obesidade                     0
Obito                         0
Outros_Fatores_De_Risco       0
Pneumopatia                   0
Puérpera                      0
Síndrome_De_Down              0
dtype: int64


### Verificação do número de linhas duplicadas

In [22]:
linhas_duplicadas = dados.duplicated().sum()
print("\nNúmero de linhas duplicadas:", linhas_duplicadas)


Número de linhas duplicadas: 4743


### Verificação de dados sujos

In [23]:
def verificar_padrao_data(data):
    if pd.isnull(data):
        return False
    
    padrao = re.compile(r'^\d{2}/\d{2}/\d{4}$')
    if re.match(padrao, str(data)):
        return True
    else:
        return False
    
# Verificação se é uma data suja
datas_sujas = []
for index, row in dados.iterrows():
    if not verificar_padrao_data(row['Data_Inicio_Sintomas']):
        datas_sujas.append((index, row['Data_Inicio_Sintomas']))

### Estatísticas descritivas

In [24]:
print("\nEstatísticas descritivas:")
estatisticas_descritivas = dados.describe()
print(estatisticas_descritivas)


Estatísticas descritivas:
              Idade          Obito
count  96802.000000  100000.000000
mean      41.089347       0.026390
std       18.432236       0.160293
min        0.000000       0.000000
25%       28.000000       0.000000
50%       40.000000       0.000000
75%       54.000000       0.000000
max      120.000000       1.000000


### Calcular o IQR e identificar outliers

In [25]:
Q1 = dados['Idade'].quantile(0.25)
Q3 = dados['Idade'].quantile(0.75)
IQR = Q3 - Q1

outliers = dados[(dados['Idade'] < Q1 - 1.5 * IQR) | (dados['Idade'] > Q3 + 1.5 * IQR)]
outliers.to_csv('outliers.csv', index=False)

## Cross Validation com 7 algoritmos

### Exclusão de linhas com valores faltantes

In [26]:
dados.dropna(inplace=True)

### Seleção de atributos (x) e variável alvo (y)

In [27]:
X = dados[['Asma', 'Cardiopatia', 'Data_Inicio_Sintomas', 'Diabetes', 'Diagnostico_Covid19', 'Doenca_Hematologica',
           'Doenca_Hepatica', 'Doenca_Neurologica', 'Doenca Renal', 'Genero', 'Idade', 'Imunodepressao', 'Municipio',
           'Obesidade', 'Outros_Fatores_De_Risco', 'Pneumopatia', 'Puérpera', 'Síndrome_De_Down']].copy()
y = dados['Obito'].copy()

### Normalização dos dados

In [28]:
le = LabelEncoder()

for column in X.columns:
    if X[column].dtype == 'object':
        X[column] = le.fit_transform(X[column].astype(str))

scaler = MinMaxScaler()

X_normalizado = scaler.fit_transform(X)

### Lista de classificadores

In [29]:
classifiers = [
    GaussianNB(),
    KNeighborsClassifier(),
    LogisticRegression(),
    XGBClassifier(),
    MultinomialNB(),
    RidgeClassifier(),
    NearestCentroid()
]

### Dicionário para armazenar as métricas de cada classificador

In [30]:
metrics = {
    'Acurácia': [],
    'Precisão': [],
    'Sensibilidade/Recall': [],
    'Medida-F1': []
}

### Execução da cross validation para cada classificador

In [31]:
for classifier in classifiers:
    # Calcula a acurácia usando validação cruzada (cv=10 para 10 folds)
    accuracy = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='accuracy').mean()
    # Calcula a precisão usando validação cruzada
    precision = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='precision').mean()
    # Calcula a sensibilidade/recall usando validação cruzada
    recall = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='recall').mean()
    # Calcula a medida-F1 usando validação cruzada
    f1 = cross_val_score(classifier, X_normalizado, y, cv=10, scoring='f1').mean()

    metrics['Acurácia'].append(accuracy)
    metrics['Precisão'].append(precision)
    metrics['Sensibilidade/Recall'].append(recall)
    metrics['Medida-F1'].append(f1)

### Exibição dos resultados

In [32]:
for metric, values in metrics.items():
    print(f'{metric}: {values}')

Acurácia: [0.9518238730303962, 0.9707061003042508, 0.9726887644306048, 0.9718894438989597, 0.9616750198336668, 0.9727510578656025, 0.9490522531913979]
Precisão: [0.3144547100142251, 0.41755596424579655, 0.504293983422456, 0.46862853442151203, 0.35139280722466604, 0.5656516290726816, 0.29235273693094294]
Sensibilidade/Recall: [0.6422701348081575, 0.17771056573337946, 0.19932307869570226, 0.1936326189653186, 0.4721468487152897, 0.0287965203364443, 0.6058949763797673]
Medida-F1: [0.4220895665639242, 0.24902914507367013, 0.28552040912992654, 0.2732904625631137, 0.40281810335436063, 0.054719811649583214, 0.3943618014833987]
